# Data Manipulation in Python (CS2006 P2)

On the 12th of November 2014, the European Space Agency lander Philae made the first ever soft landing of a spacecraft on the surface of a comet, 67P/Churyumov-Gerasimenko, having been carried there by the probe Rosetta. The news of the acheivement was disseminated through various social media platforms over the following days and weeks, including Twitter. This notebook analyses data independently gathered from Twitter to track the volume and nature of user activity related to the landing over the period of 3 weeks after the landing.

In [1]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt;
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import operator
plotly.tools.set_credentials_file(username='ir47', api_key='k0qjUbd6owGLygMXJPZ5')

ImportError: No module named wordcloud

In [2]:
df=pd.read_csv("../data/CometLanding.csv",encoding="UTF-8")

In [3]:
len(df)

77319

In [4]:
df.drop_duplicates(['id_str'],inplace = True)

In [5]:
numTweets = len(df)

The raw data contained some duplicate tweets, which were removed, and the number of total remaining unique tweets is displayed below: 

In [6]:
print("Total number of Tweets: " + str(numTweets))

Total number of Tweets: 77268


In [7]:
df = df[df['text'].notnull()]

The number of unique users is displayed below:

In [8]:
len(df['from_user'].unique())

50195

The number of tweets by language are displayed here. By far the most common is US English, with 52316 tweets.

In [9]:
language = df.groupby('user_lang')

In [10]:
language.size()

user_lang
ar           428
bg             1
ca           309
cs            42
da            89
de          2916
el            29
en         52316
en-AU          1
en-GB         23
en-gb       1972
es          7540
es-MX          2
eu            62
fa             2
fi           108
fil           10
fr          3313
gl            36
he             2
hi             2
hu            41
id            66
it          2664
ja          1514
ko            98
msa            1
nb             1
nl           838
no            36
pl           157
pt           508
pt-PT          1
ro             8
ru           794
sv           126
th            57
tr           761
uk            43
ur             1
vi             1
xx-lc         24
zh-CN          6
zh-Hans        6
zh-cn        285
zh-tw         27
dtype: int64

The majority of tweets were actually retweets, which meant that they started with the letters "RT":

In [11]:
dfNoRT = df[~df.text.str.startswith('RT', na=False)]

In [12]:
numReTweets = numTweets - len(dfNoRT) 

In [13]:
print("Total number of retweets: " + str(numReTweets))

Total number of retweets: 59999


Roughly 1/2 of the remainder are replies to earlier tweets.

In [14]:
dfReplies = df[~df.text.str.contains('@', na=False)]

In [15]:
numReplies = len(dfReplies)

In [16]:
print("Total number of replies: " + str(numReplies))

Total number of replies: 10624


Hashtags are an important 

In [17]:
import re
hashtags = []
for index, row in dfNoRT.iterrows():
    text = (row['text'].split(" "))       
    for token in text:
        re.sub('[\W_]', '', token)
        if token.startswith('#'):
            hashtags.append(str(token))



In [25]:
hashtagCount = {}

for hashtag in hashtags:
    if hashtag not in hashtagCount:
        hashtagCount[hashtag] = 1
    else:
        counter = hashtagCount.get(hashtag,'none')
        hashtagCount.update({hashtag: counter+1})
        
for key,val in hashtagCount.items():
    if val>150:
        print (repr(key) + "=>" + repr(val))
    


'#67P'=>400
'#CometLanding:'=>161
'#Rosetta'=>1471
'#Cometlanding'=>165
'#CometLanding.'=>244
'#Philae'=>734
'#rosetta'=>178
'#ESA'=>194
'#cometlanding'=>1834
'#CometLanding'=>12741
'#rosettamission'=>169
'#HappyBirthdaySandaraPark'=>928
'#WishKoSaPasko'=>929


In [21]:
words = []

for key,val in hashtagCount.items():
    words.append(key)
    
words = [e[1:] for e in words]
stopwords = set(STOPWORDS)
stopwords.add("CometLanding")


wordcloud = WordCloud(background_color='white',stopwords=stopwords,max_words=30000,max_font_size=40, random_state=42).generate(str(hashtags))
plt.figure(figsize=(10,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

NameError: name 'STOPWORDS' is not defined

In [22]:
data = [go.Bar(x=['Tweets', 'Retweets', 'Replies'],y=[numTweets,numReTweets,numReplies])]
layout = go.Layout(
    title='Number of Retweets, Replies and Tweets',
    yaxis=dict(
        title='Usage',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

NameError: name 'go' is not defined

In [23]:
dfSource = df
import re
items = []
for index, row in dfSource.iterrows():
    text = (row['source'])
    for token in str(text):
        if token.endswith('>'):
            split1 = text.split("</a>")
            split2 = str(split1).split(">")
            split2 = str(split2).split(",")
            items.append(str(split2[1]))
            
appCount = {}

for device in items:
    if device not in appCount:
        appCount[device] = 1
    else:
        counter = appCount.get(device,'none')
        appCount.update({device: counter+1})

In [24]:
topItems = sorted(appCount.items(), key=operator.itemgetter(1),reverse=True)

NameError: name 'operator' is not defined

In [ ]:
#Gets the top 4 applications
topApplication = topItems[0][0]
topApplicationNum = topItems[0][1]
    
secondApplication = topItems[1][0]
secondApplicationNum = topItems[1][1]
    
thirdApplication = topItems[2][0]
thirdApplicationNum = topItems[2][1]

fourthApplication = topItems[3][0]
fourthApplicationNum = topItems[3][1]

totalApplications = sum(appCount.values())
otherApplicationNum = (totalApplications-(topApplicationNum+secondApplicationNum+thirdApplicationNum+fourthApplicationNum))


In [ ]:
labels = [topApplication,secondApplication,thirdApplication,fourthApplication," \"Other\'"]
values = [topApplicationNum,secondApplicationNum,thirdApplicationNum,fourthApplicationNum,otherApplicationNum]

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

In [31]:
dfSource = df
import re
dates = []
for index, row in dfSource.iterrows():
    text = (row['created_at'])
    dates.append(text[0:10])
    print (text)
        
len(dates)


Fri Dec 05 18:07:14 +0000 2014
Fri Dec 05 18:05:46 +0000 2014
Fri Dec 05 18:05:36 +0000 2014
Fri Dec 05 18:03:00 +0000 2014
Fri Dec 05 18:02:32 +0000 2014
Fri Dec 05 18:00:28 +0000 2014
Fri Dec 05 17:59:42 +0000 2014
Fri Dec 05 17:53:14 +0000 2014
Fri Dec 05 17:50:33 +0000 2014
Fri Dec 05 17:48:05 +0000 2014
Fri Dec 05 17:45:58 +0000 2014
Fri Dec 05 17:45:22 +0000 2014
Fri Dec 05 17:39:05 +0000 2014
Fri Dec 05 17:35:43 +0000 2014
Fri Dec 05 17:26:11 +0000 2014
Fri Dec 05 17:18:10 +0000 2014
Fri Dec 05 16:46:45 +0000 2014
Fri Dec 05 16:36:47 +0000 2014
Fri Dec 05 16:35:11 +0000 2014
Fri Dec 05 16:34:31 +0000 2014
Fri Dec 05 16:32:10 +0000 2014
Fri Dec 05 16:31:27 +0000 2014
Fri Dec 05 16:21:48 +0000 2014
Fri Dec 05 16:21:43 +0000 2014
Fri Dec 05 16:21:21 +0000 2014
Fri Dec 05 16:19:00 +0000 2014
Fri Dec 05 16:14:10 +0000 2014
Fri Dec 05 16:12:37 +0000 2014
Fri Dec 05 16:09:11 +0000 2014
Fri Dec 05 15:50:37 +0000 2014
Fri Dec 05 15:43:18 +0000 2014
Fri Dec 05 15:19:50 +0000 2014
Fri Dec 

Tue Dec 02 06:33:09 +0000 2014
Tue Dec 02 06:33:09 +0000 2014
Tue Dec 02 06:33:09 +0000 2014
Tue Dec 02 06:33:08 +0000 2014
Tue Dec 02 06:33:08 +0000 2014
Tue Dec 02 06:33:08 +0000 2014
Tue Dec 02 06:33:08 +0000 2014
Tue Dec 02 06:33:08 +0000 2014
Tue Dec 02 06:33:08 +0000 2014
Tue Dec 02 06:33:08 +0000 2014
Tue Dec 02 06:33:07 +0000 2014
Tue Dec 02 06:33:07 +0000 2014
Tue Dec 02 06:33:07 +0000 2014
Tue Dec 02 06:33:07 +0000 2014
Tue Dec 02 06:33:07 +0000 2014
Tue Dec 02 06:33:07 +0000 2014
Tue Dec 02 06:33:07 +0000 2014
Tue Dec 02 06:33:07 +0000 2014
Tue Dec 02 06:33:07 +0000 2014
Tue Dec 02 06:33:06 +0000 2014
Tue Dec 02 06:33:06 +0000 2014
Tue Dec 02 06:33:05 +0000 2014
Tue Dec 02 06:33:04 +0000 2014
Tue Dec 02 06:23:40 +0000 2014
Tue Dec 02 06:23:39 +0000 2014
Tue Dec 02 06:23:39 +0000 2014
Tue Dec 02 06:23:39 +0000 2014
Tue Dec 02 06:23:38 +0000 2014
Tue Dec 02 06:23:38 +0000 2014
Tue Dec 02 06:23:32 +0000 2014
Tue Dec 02 06:23:31 +0000 2014
Tue Dec 02 06:23:30 +0000 2014
Tue Dec 

Fri Nov 28 08:15:17 +0000 2014
Fri Nov 28 08:15:17 +0000 2014
Fri Nov 28 08:15:16 +0000 2014
Fri Nov 28 08:15:16 +0000 2014
Fri Nov 28 08:15:15 +0000 2014
Fri Nov 28 08:15:15 +0000 2014
Fri Nov 28 08:15:14 +0000 2014
Fri Nov 28 08:15:13 +0000 2014
Fri Nov 28 08:15:12 +0000 2014
Fri Nov 28 08:13:50 +0000 2014
Fri Nov 28 08:10:08 +0000 2014
Fri Nov 28 08:06:41 +0000 2014
Fri Nov 28 08:03:50 +0000 2014
Fri Nov 28 07:54:34 +0000 2014
Fri Nov 28 07:48:50 +0000 2014
Fri Nov 28 07:44:57 +0000 2014
Fri Nov 28 07:41:17 +0000 2014
Fri Nov 28 07:35:06 +0000 2014
Fri Nov 28 07:28:14 +0000 2014
Fri Nov 28 07:17:33 +0000 2014
Fri Nov 28 07:05:57 +0000 2014
Fri Nov 28 04:54:34 +0000 2014
Fri Nov 28 04:42:25 +0000 2014
Fri Nov 28 04:22:59 +0000 2014
Fri Nov 28 04:00:44 +0000 2014
Fri Nov 28 03:39:43 +0000 2014
Fri Nov 28 03:39:41 +0000 2014
Fri Nov 28 03:39:38 +0000 2014
Fri Nov 28 03:38:40 +0000 2014
Fri Nov 28 03:38:40 +0000 2014
Fri Nov 28 03:38:40 +0000 2014
Fri Nov 28 03:38:40 +0000 2014
Fri Nov 

Wed Nov 12 23:01:42 +0000 2014
Wed Nov 12 23:01:42 +0000 2014
Wed Nov 12 23:01:41 +0000 2014
Wed Nov 12 23:01:41 +0000 2014
Wed Nov 12 23:01:40 +0000 2014
Wed Nov 12 23:01:40 +0000 2014
Wed Nov 12 23:01:40 +0000 2014
Wed Nov 12 23:01:39 +0000 2014
Wed Nov 12 23:01:39 +0000 2014
Wed Nov 12 23:01:38 +0000 2014
Wed Nov 12 23:01:37 +0000 2014
Wed Nov 12 23:01:37 +0000 2014
Wed Nov 12 23:01:37 +0000 2014
Wed Nov 12 23:01:37 +0000 2014
Wed Nov 12 23:01:36 +0000 2014
Wed Nov 12 23:01:36 +0000 2014
Wed Nov 12 23:01:36 +0000 2014
Wed Nov 12 23:01:35 +0000 2014
Wed Nov 12 23:01:35 +0000 2014
Wed Nov 12 23:01:35 +0000 2014
Wed Nov 12 23:01:35 +0000 2014
Wed Nov 12 23:01:35 +0000 2014
Wed Nov 12 23:01:34 +0000 2014
Wed Nov 12 23:01:34 +0000 2014
Wed Nov 12 23:01:34 +0000 2014
Wed Nov 12 23:01:34 +0000 2014
Wed Nov 12 23:01:34 +0000 2014
Wed Nov 12 23:01:34 +0000 2014
Wed Nov 12 23:01:33 +0000 2014
Wed Nov 12 23:01:33 +0000 2014
Wed Nov 12 23:01:32 +0000 2014
Wed Nov 12 23:01:32 +0000 2014
Wed Nov 

Wed Nov 12 22:49:53 +0000 2014
Wed Nov 12 22:49:51 +0000 2014
Wed Nov 12 22:49:51 +0000 2014
Wed Nov 12 22:49:51 +0000 2014
Wed Nov 12 22:49:51 +0000 2014
Wed Nov 12 22:49:51 +0000 2014
Wed Nov 12 22:49:50 +0000 2014
Wed Nov 12 22:49:49 +0000 2014
Wed Nov 12 22:49:48 +0000 2014
Wed Nov 12 22:49:48 +0000 2014
Wed Nov 12 22:49:47 +0000 2014
Wed Nov 12 22:49:47 +0000 2014
Wed Nov 12 22:49:47 +0000 2014
Wed Nov 12 22:49:46 +0000 2014
Wed Nov 12 22:49:46 +0000 2014
Wed Nov 12 22:49:46 +0000 2014
Wed Nov 12 22:49:46 +0000 2014
Wed Nov 12 22:49:46 +0000 2014
Wed Nov 12 22:49:45 +0000 2014
Wed Nov 12 22:49:45 +0000 2014
Wed Nov 12 22:49:43 +0000 2014
Wed Nov 12 22:49:42 +0000 2014
Wed Nov 12 22:49:42 +0000 2014
Wed Nov 12 22:49:42 +0000 2014
Wed Nov 12 22:49:41 +0000 2014
Wed Nov 12 22:49:41 +0000 2014
Wed Nov 12 22:49:41 +0000 2014
Wed Nov 12 22:49:39 +0000 2014
Wed Nov 12 22:49:38 +0000 2014
Wed Nov 12 22:49:38 +0000 2014
Wed Nov 12 22:49:37 +0000 2014
Wed Nov 12 22:49:37 +0000 2014
Wed Nov 

...

Wed Nov 12 09:40:21 +0000 2014
Wed Nov 12 09:40:21 +0000 2014
Wed Nov 12 09:40:20 +0000 2014
Wed Nov 12 09:40:19 +0000 2014
Wed Nov 12 09:40:19 +0000 2014
Wed Nov 12 09:40:19 +0000 2014
Wed Nov 12 09:40:19 +0000 2014
Wed Nov 12 09:40:19 +0000 2014
Wed Nov 12 09:40:19 +0000 2014
Wed Nov 12 09:40:19 +0000 2014
Wed Nov 12 09:40:18 +0000 2014
Wed Nov 12 09:40:17 +0000 2014
Wed Nov 12 09:40:17 +0000 2014
Wed Nov 12 09:40:17 +0000 2014
Wed Nov 12 09:40:16 +0000 2014
Wed Nov 12 09:40:16 +0000 2014
Wed Nov 12 09:40:15 +0000 2014
Wed Nov 12 09:40:15 +0000 2014
Wed Nov 12 09:40:15 +0000 2014
Wed Nov 12 09:40:14 +0000 2014
Wed Nov 12 09:40:14 +0000 2014
Wed Nov 12 09:40:13 +0000 2014
Wed Nov 12 09:40:13 +0000 2014
Wed Nov 12 09:40:13 +0000 2014
Wed Nov 12 09:40:12 +0000 2014
Wed Nov 12 09:40:12 +0000 2014
Wed Nov 12 09:40:11 +0000 2014
Wed Nov 12 09:40:11 +0000 2014
Wed Nov 12 09:40:11 +0000 2014
Wed Nov 12 09:40:10 +0000 2014
Wed Nov 12 09:40:10 +0000 2014
Wed Nov 12 09:40:10 +0000 2014
Wed Nov 

Wed Nov 12 09:27:05 +0000 2014
Wed Nov 12 09:27:05 +0000 2014
Wed Nov 12 09:27:04 +0000 2014
Wed Nov 12 09:27:04 +0000 2014
Wed Nov 12 09:27:04 +0000 2014
Wed Nov 12 09:27:04 +0000 2014
Wed Nov 12 09:27:04 +0000 2014
Wed Nov 12 09:27:02 +0000 2014
Wed Nov 12 09:27:02 +0000 2014
Wed Nov 12 09:27:02 +0000 2014
Wed Nov 12 09:27:02 +0000 2014
Wed Nov 12 09:27:02 +0000 2014
Wed Nov 12 09:27:01 +0000 2014
Wed Nov 12 09:27:01 +0000 2014
Wed Nov 12 09:27:00 +0000 2014
Wed Nov 12 09:26:59 +0000 2014
Wed Nov 12 09:26:58 +0000 2014
Wed Nov 12 09:26:58 +0000 2014
Wed Nov 12 09:26:57 +0000 2014
Wed Nov 12 09:26:57 +0000 2014
Wed Nov 12 09:26:57 +0000 2014
Wed Nov 12 09:26:56 +0000 2014
Wed Nov 12 09:26:56 +0000 2014
Wed Nov 12 09:26:55 +0000 2014
Wed Nov 12 09:26:55 +0000 2014
Wed Nov 12 09:26:55 +0000 2014
Wed Nov 12 09:26:55 +0000 2014
Wed Nov 12 09:26:54 +0000 2014
Wed Nov 12 09:26:54 +0000 2014
Wed Nov 12 09:26:53 +0000 2014
Wed Nov 12 09:26:53 +0000 2014
Wed Nov 12 09:26:53 +0000 2014
Wed Nov 

Wed Nov 12 09:17:30 +0000 2014
Wed Nov 12 09:17:29 +0000 2014
Wed Nov 12 09:17:29 +0000 2014
Wed Nov 12 09:17:28 +0000 2014
Wed Nov 12 09:17:28 +0000 2014
Wed Nov 12 09:17:28 +0000 2014
Wed Nov 12 09:17:27 +0000 2014
Wed Nov 12 09:17:27 +0000 2014
Wed Nov 12 09:17:27 +0000 2014
Wed Nov 12 09:17:27 +0000 2014
Wed Nov 12 09:17:27 +0000 2014
Wed Nov 12 09:17:27 +0000 2014
Wed Nov 12 09:17:27 +0000 2014
Wed Nov 12 09:17:26 +0000 2014
Wed Nov 12 09:17:26 +0000 2014
Wed Nov 12 09:17:26 +0000 2014
Wed Nov 12 09:17:26 +0000 2014
Wed Nov 12 09:17:26 +0000 2014
Wed Nov 12 09:17:25 +0000 2014
Wed Nov 12 09:17:25 +0000 2014
Wed Nov 12 09:17:25 +0000 2014
Wed Nov 12 09:17:25 +0000 2014
Wed Nov 12 09:17:25 +0000 2014
Wed Nov 12 09:17:24 +0000 2014
Wed Nov 12 09:17:24 +0000 2014
Wed Nov 12 09:17:24 +0000 2014
Wed Nov 12 09:17:24 +0000 2014
Wed Nov 12 09:17:24 +0000 2014
Wed Nov 12 09:17:24 +0000 2014
Wed Nov 12 09:17:23 +0000 2014
Wed Nov 12 09:17:23 +0000 2014
Wed Nov 12 09:17:23 +0000 2014
Wed Nov 

Wed Nov 12 09:12:04 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:03 +0000 2014
Wed Nov 12 09:12:02 +0000 2014
Wed Nov 12 09:12:02 +0000 2014
Wed Nov 12 09:12:02 +0000 2014
Wed Nov 12 09:12:02 +0000 2014
Wed Nov 12 09:12:01 +0000 2014
Wed Nov 12 09:12:01 +0000 2014
Wed Nov 12 09:12:01 +0000 2014
Wed Nov 12 09:12:01 +0000 2014
Wed Nov 12 09:12:01 +0000 2014
Wed Nov 12 09:12:01 +0000 2014
Wed Nov 12 09:12:01 +0000 2014
Wed Nov 12 09:12:00 +0000 2014
Wed Nov 12 09:12:00 +0000 2014
Wed Nov 12 09:12:00 +0000 2014
Wed Nov 12 09:12:00 +0000 2014
Wed Nov 12 09:12:00 +0000 2014
Wed Nov 12 09:11:59 +0000 2014
Wed Nov 12 09:11:59 +0000 2014
Wed Nov 12 09:11:59 +0000 2014
Wed Nov 12 09:11:58 +0000 2014
Wed Nov 

77267

In [30]:
dateCount = {}
counter =0

for date in dates:
    if date not in dateCount:
        dateCount[date] = 1
    else:
        counter = dateCount.get(date,'none')
        dateCount.update({date: counter+1})

In [28]:
values =[]
dateKeys = []
for key,val in dateCount.items():
    values.append(val)
    dateKeys.append(str(key))
    print("Date: "+ str(key) + " Number of Tweets: " + str(val))
    


Date: Wed Nov 26 Number of Tweets: 400
Date: Fri Nov 28 Number of Tweets: 711
Date: Mon Dec 01 Number of Tweets: 603
Date: Sun Nov 30 Number of Tweets: 343
Date: Thu Nov 27 Number of Tweets: 497
Date: Wed Nov 12 Number of Tweets: 73212
Date: Wed Dec 03 Number of Tweets: 311
Date: Tue Dec 02 Number of Tweets: 475
Date: Fri Dec 05 Number of Tweets: 87
Date: Thu Dec 04 Number of Tweets: 200
Date: Sat Nov 29 Number of Tweets: 428
